# Tool Call Example

In [ ]:
from datetime import datetime, timedelta
import json
import random
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
model = "qwen/qwen3-4b-2507"

def get_delivery_date(order_id:str):
    """Generate a random delivery date between today and 14 days from now
    In a real-world scenario, this function would query a database or API"""
    today = datetime.now()
    random_days = random.randint(1, 14)
    delivery_date = today + timedelta(days=random_days)
    print(f"\nget_delivery_date function returns delivery date: \n\n{delivery_date}",
          flush=True)
    return delivery_date

tools = [
    {
        "type":"function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters":  {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required":["order_id"],
                "additionalProperties": False,
            },
        },
    }
]

messages = [
    {
        "role":"system",
        "content":"You are a helpful customer support assistant. Use the supplied tools to assist the user.",
    },
    {
        "role": "user",
        "content": "Give me the delivery date and time for order number 1017",
    },
]

# LM Studio
response = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
)

print('\nModel response requesting tool call:\n', flush=True)
print(response, flush=True)

# Extract the arguments for get_delivery_date
# Note this code assumes we have already determined that the model generated a function call.
tool_call = response.choices[0].message.tool_calls[0]
arguments = json.loads(tool_call.function.arguments)

order_id = arguments.get("order_id")

# Call the get_delivery_date function with the extracted order_id
delivery_date = get_delivery_date(order_id)

assistant_tool_call_request_message = {
    "role": "assistant",
    "tool_calls": [
        {
            "id":response.choices[0].message.tool_calls[0].id,
            "type":response.choices[0].message.tool_calls[0].type,
            "function":response.choices[0].message.tool_calls[0].function,
        }
    ],
}

# Create a message containing the result of the function call
function_call_result_message = {
    "role":"tool",
    "content": json.dumps(
        {
            "order_id": order_id,
            "delivery_date": delivery_date.strftime("%Y-%m-%d %H:%M:%S"),
        }
    ),
    "tool_call_id": response.choices[0].message.tool_calls[0].id,
}

# Prepare the chat completion call payload
completion_messages_payload = [
    messages[0],
    messages[1],
    assistant_tool_call_request_message,
    function_call_result_message,
]

# Call the OpenAI API's chat completions endpoint to send the tool call result back
# to the model
response = client.chat.completions.create(
    model=model,
    messages=completion_messages_payload,
)

print("\nFinal model response with knowledge of the tool call result:\n")
print(response.choices[0].message.content, flush=True)


Model response requesting tool call:

ChatCompletion(id='chatcmpl-ijy69c2j37j9mngd0274a', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='478634966', function=Function(arguments='{"order_id":"1017"}', name='get_delivery_date'), type='function')]))], created=1757079674, model='qwen/qwen3-4b-2507', object='chat.completion', service_tier=None, system_fingerprint='qwen/qwen3-4b-2507', usage=CompletionUsage(completion_tokens=25, prompt_tokens=223, total_tokens=248, completion_tokens_details=None, prompt_tokens_details=None), stats={})

get_delivery_date function returns delivery date: 

2025-09-12 07:41:15.031488


# F1 Tool Calls

### Functions

In [26]:
def get_f1_driver_standings(standings_type:str)->str:
    """Get F1 2025 driver standings."""
    if standings_type=="team":
        url = f"https://www.formula1.com/en/results/2025/team"
    else:
        url = f"https://www.formula1.com/en/results/2025/drivers"
    soup_og = extract_text_from_dynamic_site(url, wait_time=3)
    html_table = soup_og.find_all('table')[0]
    html_text = str(html_table.prettify())
    html_text = re.sub(" class=\"[^\"]*\"", "", html_text)
    html_text = re.sub("<img[^>]*>", "", html_text)
    html_text = re.sub("<a[^>]*>", "", html_text)
    html_text = re.sub("<span[^>]*>", "", html_text)
    html_text = re.sub("</span[^>]*>", "", html_text)
    html_text = re.sub("<br>", "", html_text)
    html_text = re.sub("<p>", "", html_text)
    html_text = re.sub("\\\n *", "", html_text)
    drivers_standings_table = htmltabletomd.convert_table(html_text)
    return drivers_standings_table

def get_f1_team_standings():
    """Get F1 2025 team standings."""
    url = f"https://www.formula1.com/en/results/2025/team"
    soup_og = extract_text_from_dynamic_site(url, wait_time=3)
    html_table = soup_og.find_all('table')[0]
    html_text = str(html_table.prettify())
    html_text = re.sub(" class=\"[^\"]*\"", "", html_text)
    html_text = re.sub("<img[^>]*>", "", html_text)
    html_text = re.sub("<a[^>]*>", "", html_text)
    html_text = re.sub("<span[^>]*>", "", html_text)
    html_text = re.sub("</span[^>]*>", "", html_text)
    html_text = re.sub("<br>", "", html_text)
    html_text = re.sub("<p>", "", html_text)
    html_text = re.sub("\\\n *", "", html_text)
    standings_table = htmltabletomd.convert_table(html_text)
    return standings_table

In [28]:
tbl = get_f1_driver_standings('drivers')

Loaded: https://www.formula1.com/en/results/2025/drivers


In [18]:
# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
model = "qwen/qwen3-4b-2507"

tools = [
    {
        "type":"function",
        "function": {
            "name": "get_f1_standings",
            "description": "Get the current F1 drivers standings. Use it to determine the points for each driver in the driver's championship.",
            "parameters":  {
                "type": "object",
                "properties": {
                    "standings_type": {
                        "type": "string",
                        "description": "Selects either 'drivers' or 'team' championship standings."
                    }
                }
            },
                "required":["standings_type"],
                "additionalProperties": False,
            },
        },
]

messages = [
    {
        "role":"system",
        "content":"You are an expert in Formula 1 that will assist the user with questions regarding the sport. Use the supplied tools to assist the user.",
    },
    {
        "role": "user",
        "content": "Which driver who is NOT part of Red Bull, Mercedes, Ferrari or Mclaren has the most points?",
    },
]

# LM Studio
response = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
)

print('\nModel response requesting tool call:\n', flush=True)
print(response, flush=True)

# Extract the arguments for get_delivery_date
# Note this code assumes we have already determined that the model generated a function call.
tool_call = response.choices[0].message.tool_calls[0]
arguments = json.loads(tool_call.function.arguments)

standings_type = arguments.get("standings_type")
# Call the get_delivery_date function with the extracted order_id
drivers_standings_table = get_f1_driver_standings(standings_type)

assistant_tool_call_request_message = {
    "role": "assistant",
    "tool_calls": [
        {
            "id":response.choices[0].message.tool_calls[0].id,
            "type":response.choices[0].message.tool_calls[0].type,
            "function":response.choices[0].message.tool_calls[0].function,
        }
    ],
}

# Create a message containing the result of the function call
function_call_result_message = {
    "role":"tool",
    "content": json.dumps(
        {
            "standings_type": standings_type,
            "drivers_standings_table": drivers_standings_table,
        }
    ),
    "tool_call_id": response.choices[0].message.tool_calls[0].id,
}

# Prepare the chat completion call payload
completion_messages_payload = [
    messages[0],
    messages[1],
    assistant_tool_call_request_message,
    function_call_result_message,
]

# Call the OpenAI API's chat completions endpoint to send the tool call result back
# to the model
response = client.chat.completions.create(
    model=model,
    messages=completion_messages_payload,
)

print("\nFinal model response with knowledge of the tool call result:\n")
print(response.choices[0].message.content, flush=True)


Model response requesting tool call:

ChatCompletion(id='chatcmpl-dmni5wo1ty9updbcwkusaj', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='549168276', function=Function(arguments='{"standings_type":"drivers"}', name='get_f1_standings'), type='function')]))], created=1757082163, model='qwen/qwen3-4b-2507', object='chat.completion', service_tier=None, system_fingerprint='qwen/qwen3-4b-2507', usage=CompletionUsage(completion_tokens=25, prompt_tokens=229, total_tokens=254, completion_tokens_details=None, prompt_tokens_details=None), stats={})
Loaded: https://www.formula1.com/en/results/2025/drivers

Final model response with knowledge of the tool call result:

The driver who is NOT part of Red Bull, Mercedes, Ferrari, or McLaren and has the most points is **Alexander Albon** from Williams, with